<a href="https://colab.research.google.com/github/imane-kasmi/TP1__NLP/blob/main/tp1nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'customer-support-on-twitter:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4133%2F8841%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240224%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240224T175236Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6b6dd4a796220d53ab860c6bda9188e93847487fa90896bd62d9ba77635175baf468d9f67de16af0c686c1877e535b74a046bce40d43b8e7072ec9930ed7d59769d670cacc4771b738bd625b095b50cf6fb7c23c46397905f7026f393506cadb0f53c5df7e4767b2db83a81488167dc43e161ee33731e86f83ec586cca94ebb8bd20435d53dab8d6e4fa8a28d69b2d7c0d980e1154ecf9cf6feda25aacbf791c3cd8508df9e78b4e3003d6d5a2c844bae440d1b5bbccf36cf686f9841523377746ba8b519cc0d1ea2112d81261ba103e35803692f49915dc54e5446f2d82ff74380f28632605676c79bb1d8f0d91eb79d5aa22bb248bb92a3cf8f28ccf4c36d1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 176772673 bytes downloaded
Downloaded and uncompressed: customer-support-on-twitter
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-support-on-twitter/sample.csv
/kaggle/input/customer-support-on-twitter/twcs/twcs.csv


In [5]:
# Affichage de la colonne text_cleaned avant nettoyage
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

full_df = pd.read_csv("/kaggle/input/customer-support-on-twitter/twcs/twcs.csv", nrows=5000)
print("Avant nettoyage :")
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
full_df.head()

Avant nettoyage :


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [8]:
# mise en minuscule
df['text_cleaned'] = df['text'].str.lower()
# Affichage de la colonne text_cleaned après mise en minuscules
print("\nAprès mise en minuscules :")
print(df['text_cleaned'])


Après mise en minuscules :
0       @115712 i understand. i would like to assist y...
1           @sprintcare and how do you propose we do that
2       @sprintcare i have sent several private messag...
3       @115712 please send us a private message so th...
4                                      @sprintcare i did.
                              ...                        
4995    @117290 hm, they should definitely resume if a...
4996    @117290 that's where they need to be on, so ch...
4997    @115940 can you guys maybe like....fix your st...
4998    @117291 hulu is only available in the u.s. rig...
4999    @115940 when will the platform available in la...
Name: text_cleaned, Length: 5000, dtype: object


In [9]:
#supprression de la ponctuation
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
#affichage de text_cleaned apres suppresion de ponctuation
print (df['text_cleaned'])

0       115712 i understand i would like to assist you...
1            sprintcare and how do you propose we do that
2       sprintcare i have sent several private message...
3       115712 please send us a private message so tha...
4                                        sprintcare i did
                              ...                        
4995    117290 hm they should definitely resume if alr...
4996    117290 thats where they need to be on so check...
4997    115940 can you guys maybe likefix your stuff f...
4998    117291 hulu is only available in the us right ...
4999    115940 when will the platform available in lat...
Name: text_cleaned, Length: 5000, dtype: object


In [11]:
# Suppression des mots vides (stopwords)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
#suppression des mots frequents
from collections import Counter
n = 10  # Nombre de mots fréquents à supprimer
word_freq = Counter(' '.join(df['text_cleaned']).split())
common_words = [word for word, freq in word_freq.most_common(n)]
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in common_words]))


In [14]:
#suppression des mots rares
n = 5  # Nombre minimum d'occurrences pour conserver un mot
word_freq = Counter(' '.join(df['text_cleaned']).split())
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word_freq[word] >= n]))
# Affichage de la colonne text_cleaned après suppression des mots rares
print("\nAprès suppression des mots rares :")
print(df['text_cleaned'])


Après suppression des mots rares :
0       115712 understand would like assist would need...
1                                              sprintcare
2       sprintcare sent several private messages one r...
3       115712 send private message assist click messa...
4                                              sprintcare
                              ...                        
4995         hm definitely already enabled cc settings 12
4996              thats need check home always happens 22
4997         115940 guys maybe stuff time one episode wtf
4998    hulu available right well sure share interest ...
4999               115940 platform available specifically
Name: text_cleaned, Length: 5000, dtype: object


In [15]:
#Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))


In [17]:
#Lemmatization
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [20]:
#suppression des emojis,emoticone,URL,balises Html
!pip install emoji
import emoji
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: emoji.demojize(x))

df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'[\U00010000-\U0010ffff]', '', x))
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'http\S+|www\S+', '', x))
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'<.*?>', '', x))


In [23]:
# Ajout de la colonne 'tweet_id' avec des valeurs par défaut pour l'exemple
df['tweet_id'] = range(1, len(df) + 1)

# Réorganiser les colonnes dans l'ordre requis
df = df[['tweet_id', 'text', 'text_cleaned']]

# Affichage du dataframe final
print(df)

      tweet_id                                               text  \
0            1  @115712 I understand. I would like to assist y...   
1            2      @sprintcare and how do you propose we do that   
2            3  @sprintcare I have sent several private messag...   
3            4  @115712 Please send us a Private Message so th...   
4            5                                 @sprintcare I did.   
...        ...                                                ...   
4995      4996  @117290 Hm, they should definitely resume if a...   
4996      4997  @117290 That's where they need to be on, so ch...   
4997      4998  @115940 Can you guys maybe like....FIX your st...   
4998      4999  @117291 Hulu is only available in the U.S. rig...   
4999      5000  @115940 when will the platform available in la...   

                                           text_cleaned  
0     115712 understand would like assist would need...  
1                                             sprintcar